In [1]:
import dspy
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric

# Set up the LM.
turbo = dspy.OpenAI(model='gpt-3.5-turbo-instruct', max_tokens=250)
dspy.settings.configure(lm=turbo)

# Load math questions from the GSM8K dataset.
gsm8k = GSM8K()
gsm8k_trainset, gsm8k_devset = gsm8k.train[:10], gsm8k.dev[:10]

print(gsm8k_trainset)

ModuleNotFoundError: No module named 'dspy'

In [3]:
qa_pair_list = []
qa_pair_list.append(dspy.Example(question='''Can you create an agent for cooking?", answer="Agent Name: God King Chef.
                                 Description: A master chef.
                                 Characteristics: Mean, Precise, Tasteful.
                                 Task: Innovating on existing recipes to create cutting-edge, flavorful food that elicits euphoria.''').with_inputs("question"))
qa_pair_list.append(dspy.Example(question='''Can you tell me the best bike to buy?", answer="Agent Name: Bicyclist.
                                 Description: Knows everything about biking.
                                 Characteristics: Lanky, enduring, knowledgable.
                                 Task: Knows what to look for and how to look for the best bikes online.
                                 Tools: Web search''').with_inputs("question"))
qa_pair_list.append(dspy.Example(question='''What is love?", answer="Agent Name: Cupid.
                                 Description: Knows what love is.
                                 Characteristics: Lovable, empathetic, kind.
                                 Task: Tactfully describes love to people scared of heartbreak.''').with_inputs("question"))
qa_pair_list.append(dspy.Example(question='''What is the square root of 2?", answer="Agent Name: Terry Tao.
                                 Description: The best mathematician.
                                 Characteristics: Knowledgable, aware, intelligent.
                                 Task: Operates an internal calculator to answer user queries.
                                 Tools: Calculator.''').with_inputs("question"))
qa_pair_list.append(dspy.Example(question='''Woopy wob bop?", answer="Agent Name: Abstraction.
                                 Description: Random object generator.
                                 Characteristics: Esoteric, eccentric, exciting.
                                 Task: Uses pure abstraction to understand the user query and respond to it in equally convoluted language.''').with_inputs("question"))

NameError: name 'dspy' is not defined

In [ ]:
class FactJudge(dspy.Signature):
    """Judge if the answer is correct based on typical AI agent design and construction."""

    question = dspy.InputField(desc="Question to be answered")
    answer = dspy.InputField(desc="Answer for the question")
    factually_correct = dspy.OutputField(desc="Is the answer factually correct based on the context?", prefix="Factual[Yes/No]:")

judge = dspy.ChainOfThought(FactJudge)

def factuality_metric(example, pred):
    factual = judge(question=example.question, answer=pred.answer)
    return int(factual=="Yes")

In [ ]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")
    
    def forward(self, question):
        return self.prog(question=question)
    
    from dspy.teleprompt import BootstrapFewShot

# Set up the optimizer: we want to "bootstrap" (i.e., self-generate) 4-shot examples of our CoT program.
config = dict(max_bootstrapped_demos=4, max_labeled_demos=4)

# Optimize! Use the `gsm8k_metric` here. In general, the metric is going to tell the optimizer how well it's doing.
teleprompter = BootstrapFewShot(metric=gsm8k_metric, **config)
optimized_cot = teleprompter.compile(CoT(), trainset=gsm8k_trainset)

from dspy.evaluate import Evaluate

# Set up the evaluator, which can be used multiple times.
evaluate = Evaluate(devset=gsm8k_devset, metric=factuality_metric, num_threads=4, display_progress=True, display_table=0)

# Evaluate our `optimized_cot` program.
evaluate(optimized_cot)

In [ ]:
turbo.inspect_history(n=1)

In [ ]:
optimized_cot(question='Create a framework for specific AI assistant agents. Make sure the agent can search the web. Use the most up to date code and techniques.')